<a href="https://colab.research.google.com/github/ajachy/hello-world/blob/master/ricediseasegithub20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [66]:
import os
import zipfile

local_zip = 'validation_ricedisease.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r') 
zip_ref.extractall('/tmp/validation_ricedisease')

zip_ref.close()

local_zip = 'ricedisease.zip'
zip_r = zipfile.ZipFile(local_zip, 'r')
zip_r.extractall('/tmp/ricedisease')
 


In [67]:
# Variables for complete directory path

train_blb = os.path.join('/tmp/ricedisease/bacterial_leaf_blight')

train_blast = os.path.join('/tmp/ricedisease/blast')

train_bs = os.path.join('/tmp/ricedisease/brownspot')

validation_blb = os.path.join('/tmp/validation_ricedisease/bacterial_leaf_blight')

validation_blast = os.path.join('/tmp/validation_ricedisease/blast')

validation_bs = os.path.join('/tmp/validation_ricedisease/brownspot')

 

In [68]:
print('total bacterial_leaf_blight images:', len(os.listdir(train_blb)))
print('total blast images:', len(os.listdir(train_blast)))
print('total brownspot images:', len(os.listdir(train_bs)))

print('total validation bacterial_leaf_blightt images:', len(os.listdir(validation_blb)))
print('total validation blast images:', len(os.listdir(validation_blast)))
print('total  validation brownspot images:', len(os.listdir(validation_bs)))


total bacterial_leaf_blight images: 76
total blast images: 60
total brownspot images: 80
total validation bacterial_leaf_blightt images: 20
total validation blast images: 20
total  validation brownspot images: 20


In [69]:
import tensorflow as tf

In [70]:
model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 300x300 with 3 bytes color
    # This is the first convolution
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(300, 300, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
     
    # The fourth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fifth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

   

    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),

    
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')

 
])

 

In [71]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_25 (Conv2D)           (None, 298, 298, 16)      448       
_________________________________________________________________
max_pooling2d_25 (MaxPooling (None, 149, 149, 16)      0         
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 147, 147, 32)      4640      
_________________________________________________________________
max_pooling2d_26 (MaxPooling (None, 73, 73, 32)        0         
_________________________________________________________________
conv2d_27 (Conv2D)           (None, 71, 71, 64)        18496     
_________________________________________________________________
max_pooling2d_27 (MaxPooling (None, 35, 35, 64)        0         
_________________________________________________________________
conv2d_28 (Conv2D)           (None, 33, 33, 64)       

In [76]:

from tensorflow.keras.optimizers import RMSprop

model.compile(loss='categorical_crossentropy',
              optimizer= RMSprop(lr=0.001),
              #optimizer='adam',
              metrics=['accuracy'])

In [77]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(rescale=1/255)
validation_datagen = ImageDataGenerator(rescale=1/255)

# Flow training images in batches of 128 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        directory = '/tmp/ricedisease/',  # This is the source directory for training images
        target_size=(300, 300),  # All images will be resized to 300x300
        batch_size= 27,
    
        class_mode='categorical')

# Flow training images in batches of 128 using train_datagen generator
validation_generator = validation_datagen.flow_from_directory(
        directory = '/tmp/validation_ricedisease/',  # This is the source directory for training images
        target_size=(300, 300),  # All images will be resized to 300x300
         batch_size=7,
    
        class_mode='categorical')

Found 216 images belonging to 3 classes.
Found 60 images belonging to 3 classes.


In [78]:

from keras.callbacks import *
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('val_accuracy')>0.6 and logs.get('accuracy')>0.7 ):
      print("\nReached 60% accuracy so cancelling training!")
      self.model.stop_training = True

    

In [86]:


history = model.fit(
          train_generator,
          steps_per_epoch= 8,    
          validation_data=validation_generator,
          validation_steps= 8,
          epochs=20,
          verbose=1,
          shuffle=True,
          callbacks= myCallback()
         )

Epoch 1/20
8/8 [==============================] - 1s 127ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 10.8369 - val_accuracy: 0.5179
Epoch 2/20
8/8 [==============================] - 1s 123ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 10.1126 - val_accuracy: 0.5536
Epoch 3/20
8/8 [==============================] - 1s 122ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 10.1825 - val_accuracy: 0.5179
Epoch 4/20
8/8 [==============================] - 1s 123ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 10.9690 - val_accuracy: 0.5179
Epoch 5/20
8/8 [==============================] - 1s 121ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 10.0004 - val_accuracy: 0.5357
Epoch 6/20
8/8 [==============================] - 1s 121ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 10.1912 - val_accuracy: 0.5179
Epoch 7/20
8/8 [==============================] - 1s 121ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 10.0028 - val_ac

In [88]:
import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():
 
  # predicting images
  path = '/content/' + fn
  img = image.load_img(path, target_size=(300, 300))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images)
  print(classes)

  max = classes.argmax()

if max == 0 :
      print(fn + " is  having  bacterial_leaf_blight")
elif  max == 1:
     print(fn + " is  having  blast")
elif max == 2:
     print(fn + " is  having  brownspot")
 

 

Saving bs.jpg to bs (2).jpg
[[0. 0. 1.]]
bs.jpg is  having  brownspot
